# Project 5: NLP on Financial Statements
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `project_helper` and `project_tests`. These are custom packages built to help you solve the problems.  The `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.

### Install Packages

In [11]:
import sys
!{sys.executable} -m pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


### Load Packages

In [12]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper
import project_tests
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By

### Download NLP Corpora
You'll need two corpora to run this project: the stopwords corpus for removing stopwords and wordnet for lemmatizing.

In [13]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bwayn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bwayn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Get 10ks
We'll be running NLP analysis on 10-k documents. To do that, we first need to download the documents. For this project, we'll download 10-ks for a few companies. To lookup documents for these companies, we'll use their CIK. If you would like to run this against other stocks, we've provided the dict `additional_cik` for more stocks. However, the more stocks you try, the long it will take to run.

### Get list of 10-ks
The SEC has a limit on the number of calls you can make to the website per second. In order to avoid hiding that limit, we've created the `SecAPI` class. This will cache data from the SEC and prevent you from going over the limit.

With the class constructed, let's pull a list of filled 10-ks from the SEC for each company.

Let's pull the list using the `get_sec_data` function, then display some of the results. For displaying some of the data, we'll use Amazon as an example. 

In [14]:
from bs4 import BeautifulSoup
import requests
sec_api = project_helper.SecAPI()
cik_lookup = {
    'AMZN': '0001018724',
    'BMY': '0000014272',
    'CNP': '0001130310',
    'CVX': '0000093410',
    'FL': '0000850209',
    'FRT': '0000034903',
    'HON': '0000773840'}

example_ticker = 'AMZN'
sec_data = {'AMZN':[("https://www.sec.gov/Archives/edgar/data/1018724/000101872422000005/0001018724-22-000005-index.htm", "10-K",
                    "2019-02-01")]}

HEADER = {'Host': 'www.sec.gov', 'Connection': 'close',
          'Accept': 'application/json, text/javascript, */*; q=0.01',
          'X-Requested-With': 'XMLHttpRequest',
          'User-Agent': 'ruizhuoj@andrew.cmu.edu'
         }
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# define our parameters dictionary
param_dict = {'action':'getcompany',
              'CIK':'0001018724',
              'type':'10-k',
              'dateb':'20190101',
              'owner':'exclude',
              'start':'',
              'output':'',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict, headers=HEADER)
soup = BeautifulSoup(response.content, 'html.parser')

# Let the user know it was successful.
print('Request Successful')
print(response.url)
doc_table = soup.find_all('table', class_='tableFile2')

# define a base url that will be used for link building.
base_url_sec = r"https://www.sec.gov"

master_list = []
for cik in cik_lookup:
    for row in doc_table[0].find_all('tr'):
        # find all the columns
        cols = row.find_all('td')
        # if there are no columns move on to the next row.
        if len(cols) != 0:
            # grab the text
            filing_type = cols[0].text.strip()
            filing_date = cols[3].text.strip()
            filing_numb = cols[4].text.strip()
            # find the links
            filing_doc_href = cols[1].find('a', {'href':True, 'id':'documentsbutton'})
            filing_int_href = cols[1].find('a', {'href':True, 'id':'interactiveDataBtn'})
            filing_num_href = cols[4].find('a')
            filing_doc_link = base_url_sec + filing_doc_href['href']
            sec_data['AMZN'].append((filing_doc_link, filing_type, filing_date))
            # let the user know it's working
            print('-'*100)
            print("Filing Type: " + filing_type)
            print("Filing Date: " + filing_date)
            print("Filing Number: " + filing_numb)

pprint.pprint(sec_data[example_ticker][:5])

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001018724&type=10-k&dateb=20190101&owner=exclude&start=&output=&count=100
----------------------------------------------------------------------------------------------------
Filing Type: 10-K
Filing Date: 2018-02-02
Filing Number: 000-2251318568399
----------------------------------------------------------------------------------------------------
Filing Type: 10-K
Filing Date: 2017-02-10
Filing Number: 000-2251317588807
----------------------------------------------------------------------------------------------------
Filing Type: 10-K
Filing Date: 2016-01-29
Filing Number: 000-22513161370623
----------------------------------------------------------------------------------------------------
Filing Type: 10-K
Filing Date: 2015-01-30
Filing Number: 000-2251315560475
----------------------------------------------------------------------------------------------------
Filing Type: 10-K
Filing Date: 2014-0

### Download 10-ks
As you see, this is a list of urls. These urls point to a file that contains metadata related to each filling. Since we don't care about the metadata, we'll pull the filling by replacing the url with the filling url.

In [15]:
raw_fillings_by_ticker = {}
browser = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))
for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        print(index_url, file_type, file_date)
        if (file_type == '10-K'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')
            browser.get(file_url)
            elements = browser.find_element(By.TAG_NAME, "body")
            raw_fillings_by_ticker[ticker][file_date] = elements.text


print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

https://www.sec.gov/Archives/edgar/data/1018724/000101872418000005/0001018724-18-000005-index.htm 10-K 2018-02-02


https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/0001018724-17-000011-index.htm 10-K 2017-02-10


https://www.sec.gov/Archives/edgar/data/1018724/000101872416000172/0001018724-16-000172-index.htm 10-K 2016-01-29


https://www.sec.gov/Archives/edgar/data/1018724/000101872415000006/0001018724-15-000006-index.htm 10-K 2015-01-30


https://www.sec.gov/Archives/edgar/data/1018724/000101872414000006/0001018724-14-000006-index.htm 10-K 2014-01-31


...

https://www.sec.gov/Archives/edgar/data/1018724/000119312504029488/0001193125-04-029488-index.htm 10-K 2004-02-25


https://www.sec.gov/Archives/edgar/data/1018724/000095014903000355/0000950149-03-000355-index.htm 10-K 2003-02-19


https://www.sec.gov/Archives/edgar/data/1018724/000103221002000059/0001032210-02-000059-index.htm 10-K405 2002-01-24
https://www.sec.gov/Archives/edgar/data/1018724/000103221001500087/0001032210-01-500087-index.htm 10-K405 2001-03-23
https://www.sec.gov/Archives/edgar/data/1018724/000089102000001638/0000891020-00-001638-index.htm 10-K/A 2000-09-08
https://www.sec.gov/Archives/edgar/data/1018724/0000891020-00-000622-index.html 10-K 2000-03-29


https://www.sec.gov/Archives/edgar/data/1018724/0000891020-99-000375-index.html 10-K 1999-03-05


https://www.sec.gov/Archives/edgar/data/1018724/0000891020-98-000448-index.html 10-K405 1998-03-30
Example Document:

<SEC-DOCUMENT>0001018724-18-000005.txt : 20180202
<SEC-HEADER>0001018724-18-000005.hdr.sgml : 20180202
<ACCEPTANCE-DATETIME>20180201204115
ACCESSION NUMBER:  0001018724-18-000005
CONFORMED SUBMISSION TYPE: 10-K
PUBLIC DOCUMENT COUNT:  110
CONFORMED PERIOD OF REPORT: 20171231
FILED AS OF DATE:  20180202
DATE AS OF CHANGE:  20180201

FILER:

 COMPANY DATA: 
  COMPANY CONFORMED NAME:   AMAZON COM INC
  CENTRAL INDEX KEY:   0001018724
  STANDARD INDUSTRIAL CLASSIFICATION: RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]
  IRS NUMBER:    911646860
  STATE OF INCORPORATION:   DE
  FISCAL YEAR END:   1231

 FILING VALUES:
  FORM TYPE:  10-K
  SEC ACT:  1934 Act
  SEC FILE NUMBER: 000-22513
  FILM NUMBER:  18568399

 BUSINESS ADDRESS: 
  STREET 1:  410 TERRY AVENUE NORTH
  CITY:   SEATTLE
  STATE:   WA
  ZIP:   98109
  BUSINESS PHONE:  2062661000

 MAIL ADDRESS: 
  STREET 1:  410 TERR

### Get Documents
With theses fillings downloaded, we want to break them into their associated documents. These documents are sectioned off in the fillings with the tags `<DOCUMENT>` for the start of each document and `</DOCUMENT>` for the end of each document. There's no overlap with these documents, so each `</DOCUMENT>` tag should come after the `<DOCUMENT>` with no `<DOCUMENT>` tag in between.

Implement `get_documents` to return a list of these documents from a filling. Make sure not to include the tag in the returned document text.

In [16]:
import re


def get_documents(text):
    """
    Extract the documents from the text

    Parameters
    ----------
    text : str
        The text with the document strings inside

    Returns
    -------
    extracted_docs : list of str
        The document strings found in `text`
    """
    
    # TODO: Implement
    extracted_docs = []
    
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')   
    
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(text)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(text)]
    
    for doc_start_i, doc_end_i in zip(doc_start_is, doc_end_is):
            extracted_docs.append(text[doc_start_i:doc_end_i])
    
    return extracted_docs


project_tests.test_get_documents(get_documents)

Tests Passed


With the `get_documents` function implemented, let's extract all the documents.

In [17]:
filling_documents_by_ticker = {}

for ticker, raw_fillings in raw_fillings_by_ticker.items():
    filling_documents_by_ticker[ticker] = {}
    for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
        filling_documents_by_ticker[ticker][file_date] = get_documents(filling)


print('\n\n'.join([
    'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
    for file_date, docs in filling_documents_by_ticker[example_ticker].items()
    for doc_i, doc in enumerate(docs)][:3]))

Getting Documents from AMZN Fillings: 100%|██████████| 18/18 [00:00<00:00, 155.21filling/s]

Document 0 Filed on 2018-02-02:

<TYPE>10-K
<SEQUENCE>1
<FILENAME>amzn-20171231x10k.htm
<DESCRIPTION>10-K
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
 <head>
 ...

Document 1 Filed on 2018-02-02:

<TYPE>EX-12.1
<SEQUENCE>2
<FILENAME>amzn-20171231xex121.htm
<DESCRIPTION>EXHIBIT 12.1
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<ht...

Document 2 Filed on 2018-02-02:

<TYPE>EX-21.1
<SEQUENCE>3
<FILENAME>amzn-20171231xex211.htm
<DESCRIPTION>EXHIBIT 21.1
<TEXT>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<ht...


### Get Document Types
Now that we have all the documents, we want to find the 10-k form in this 10-k filing. Implement the `get_document_type` function to return the type of document given. The document type is located on a line with the `<TYPE>` tag. For example, a form of type "TEST" would have the line `<TYPE>TEST`. Make sure to return the type as lowercase, so this example would be returned as "test".

In [18]:
def get_document_type(doc):
    """
    Return the document type lowercased

    Parameters
    ----------
    doc : str
        The document string

    Returns
    -------
    doc_type : str
        The document type lowercased
    """
    
    # TODO: Implement
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    
    doc_type = type_pattern.findall(doc)[0][len('<TYPE>'):] 
    
    return doc_type.lower()


project_tests.test_get_document_type(get_document_type)

Tests Passed


With the `get_document_type` function, we'll filter out all non 10-k documents.

In [19]:
ten_ks_by_ticker = {}

for ticker, filling_documents in filling_documents_by_ticker.items():
    ten_ks_by_ticker[ticker] = []
    for file_date, documents in filling_documents.items():
        for document in documents:
            if get_document_type(document) == '10-k':
                ten_ks_by_ticker[ticker].append({
                    'cik': cik_lookup[ticker],
                    'file': document,
                    'file_date': file_date})


project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])

[
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2017123...
    file_date: '2018-02-02'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2016123...
    file_date: '2017-02-10'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2015123...
    file_date: '2016-01-29'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2014123...
    file_date: '2015-01-30'},
  {
    cik: '0001018724'
    file: '\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>amzn-2013123...
    file_date: '2014-01-31'},
]


## Preprocess the Data
### Clean Up
As you can see, the text for the documents are very messy. To clean this up, we'll remove the html and lowercase all the text.

In [20]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    return text


def clean_text(text):
    text = text.lower()
    text = remove_html_tags(text)
    return text

Using the `clean_text` function, we'll clean up all the documents.

In [21]:
for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
        ten_k['file_clean'] = clean_text(ten_k['file'])
        #print(ten_k['file_clean'])
    project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['file_clean'])

Cleaning AMZN 10-Ks: 100%|██████████| 18/18 [00:24<00:00,  1.36s/10-K]

[
  {
    file_clean: '\n10-k\n1\namzn-20171231x10k.htm\n10-k\n\n\n\n\n\...},
  {
    file_clean: '\n10-k\n1\namzn-20161231x10k.htm\nform 10-k\n\n\n...},
  {
    file_clean: '\n10-k\n1\namzn-20151231x10k.htm\nform 10-k\n\n\n...},
  {
    file_clean: '\n10-k\n1\namzn-20141231x10k.htm\nform 10-k\n\n\n...},
  {
    file_clean: '\n10-k\n1\namzn-20131231x10k.htm\nform 10-k\n\n\n...},
]


### Lemmatize
With the text cleaned up, it's time to distill the verbs down. Implement the `lemmatize_words` function to lemmatize verbs in the list of words provided.

In [22]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


def lemmatize_words(words):
    """
    Lemmatize words 

    Parameters
    ----------
    words : list of str
        List of words

    Returns
    -------
    lemmatized_words : list of str
        List of lemmatized words
    """
    
    # TODO: Implement
    lemmatized_words = [WordNetLemmatizer().lemmatize(word, 'v') for word in words]
    
    return lemmatized_words


project_tests.test_lemmatize_words(lemmatize_words)

Tests Passed


With the `lemmatize_words` function implemented, let's lemmatize all the data.

In [23]:
word_pattern = re.compile('\w+')

for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in tqdm(ten_ks, desc='Lemmatize {} 10-Ks'.format(ticker), unit='10-K'):
        ten_k['file_lemma'] = lemmatize_words(word_pattern.findall(ten_k['file_clean']))
        print(ten_k['file_lemma'])


Lemmatize AMZN 10-Ks:   6%|▌         | 1/18 [00:00<00:02,  8.0510-K/s]

['10', 'k', '1', 'amzn', '20171231x10k', 'htm', '10', 'k', 'document', 'unite', 'statessecurities', 'and', 'exchange', 'commissionwashington', 'd', 'c', '20549', '____________________________________form', '10', 'k____________________________________', 'mark', 'one', 'xannual', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934for', 'the', 'fiscal', 'year', 'end', 'december', '31', '2017', 'or', 'transition', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934for', 'the', 'transition', 'period', 'from', 'to', 'commission', 'file', 'no', '000', '22513____________________________________amazon', 'com', 'inc', 'exact', 'name', 'of', 'registrant', 'as', 'specify', 'in', 'its', 'charter', 'delaware', '91', '1646860', 'state', 'or', 'other', 'jurisdiction', 'ofincorporation', 'or', 'organization', 'i', 'r', 's', 'employeridentification', 'no', '410', 'terry

Lemmatize AMZN 10-Ks:  11%|█         | 2/18 [00:00<00:02,  7.7710-K/s]

['10', 'k', '1', 'amzn', '20161231x10k', 'htm', 'form', '10', 'k', 'document', 'unite', 'statessecurities', 'and', 'exchange', 'commissionwashington', 'd', 'c', '20549', '____________________________________form', '10', 'k____________________________________', 'mark', 'one', 'xannual', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934for', 'the', 'fiscal', 'year', 'end', 'december', '31', '2016', 'or', 'transition', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934for', 'the', 'transition', 'period', 'from', 'to', 'commission', 'file', 'no', '000', '22513____________________________________amazon', 'com', 'inc', 'exact', 'name', 'of', 'registrant', 'as', 'specify', 'in', 'its', 'charter', 'delaware', '91', '1646860', 'state', 'or', 'other', 'jurisdiction', 'ofincorporation', 'or', 'organization', 'i', 'r', 's', 'employeridentification', 'no', '410'

Lemmatize AMZN 10-Ks:  17%|█▋        | 3/18 [00:00<00:02,  7.2410-K/s]

...

Lemmatize AMZN 10-Ks:  33%|███▎      | 6/18 [00:00<00:01,  7.0010-K/s]


['10', 'k', '1', 'd445434d10k', 'htm', 'form', '10', 'k', 'form', '10', 'k', 'table', 'of', 'content', 'unite', 'state', 'securities', 'and', 'exchange', 'commission', 'washington', 'd', 'c', '20549', 'form', '10', 'k', 'mark', 'one', 'x', 'annual', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934', 'for', 'the', 'fiscal', 'year', 'end', 'december', '31', '2012', 'or', 'transition', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934', 'for', 'the', 'transition', 'period', 'from', 'to', 'commission', 'file', 'no', '000', '22513', 'amazon', 'com', 'inc', 'exact', 'name', 'of', 'registrant', 'as', 'specify', 'in', 'its', 'charter', 'delaware', '91', '1646860', 'state', 'or', 'other', 'jurisdiction', 'of', 'incorporation', 'or', 'organization', 'i', 'r', 's', 'employer', 'identification', 'no', '410', 'terry', 'avenue', 'north', 'seattle', 'washington

Lemmatize AMZN 10-Ks:  44%|████▍     | 8/18 [00:01<00:01,  7.4210-K/s]


['10', 'k', '1', 'd10k', 'htm', 'form', '10', 'k', 'form', '10', 'k', 'table', 'of', 'content', 'unite', 'state', 'securities', 'and', 'exchange', 'commission', 'washington', 'd', 'c', '20549', 'form', '10', 'k', 'mark', 'one', 'x', 'annual', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934', 'for', 'the', 'fiscal', 'year', 'end', 'december', '31', '2010', 'or', 'transition', 'report', 'pursuant', 'to', 'section', '13', 'or', '15', 'd', 'of', 'the', 'securities', 'exchange', 'act', 'of', '1934', 'for', 'the', 'transition', 'period', 'from', 'to', 'commission', 'file', 'no', '000', '22513', 'amazon', 'com', 'inc', 'exact', 'name', 'of', 'registrant', 'as', 'specify', 'in', 'its', 'charter', 'delaware', '91', '1646860', 'state', 'or', 'other', 'jurisdiction', 'of', 'incorporation', 'or', 'organization', 'i', 'r', 's', 'employer', 'identification', 'no', '410', 'terry', 'avenue', 'north', 'seattle', 'washington', '981

Lemmatize AMZN 10-Ks:  56%|█████▌    | 10/18 [00:01<00:01,  6.9910-K/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Remove Stopwords

In [24]:
from nltk.corpus import stopwords


lemma_english_stopwords = lemmatize_words(stopwords.words('english'))

for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in tqdm(ten_ks, desc='Remove Stop Words for {} 10-Ks'.format(ticker), unit='10-K'):
        ten_k['file_lemma'] = [word for word in ten_k['file_lemma'] if word not in lemma_english_stopwords]


print('Stop Words Removed')

Remove Stop Words for AMZN 10-Ks: 100%|██████████| 18/18 [00:00<00:00, 18.7310-K/s]

Stop Words Removed


## Analysis on 10ks
### Loughran McDonald Sentiment Word Lists
We'll be using the Loughran and McDonald sentiment word lists. These word lists cover the following sentiment:
- Negative 
- Positive
- Uncertainty
- Litigious
- Constraining
- Superfluous
- Modal

This will allow us to do the sentiment analysis on the 10-ks. Let's first load these word lists. We'll be looking into a few of these sentiments.

In [25]:
import os


sentiments = ['negative', 'positive', 'uncertainty', 'litigious', 'constraining', 'interesting']

sentiment_df = pd.read_csv(os.path.join('..', '..', 'data', 'project_5_loughran_mcdonald', 'loughran_mcdonald_master_dic_2016.csv'))
sentiment_df.columns = [column.lower() for column in sentiment_df.columns] # Lowercase the columns for ease of use

# Remove unused information
sentiment_df = sentiment_df[sentiments + ['word']]
sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
sentiment_df = sentiment_df[(sentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())
sentiment_df = sentiment_df.drop_duplicates('word')


sentiment_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\data\\project_5_loughran_mcdonald\\loughran_mcdonald_master_dic_2016.csv'

### Bag of Words
using the sentiment word lists, let's generate sentiment bag of words from the 10-k documents. Implement `get_bag_of_words` to generate a bag of words that counts the number of sentiment words in each doc. You can ignore words that are not in `sentiment_words`.

In [ ]:
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer


def get_bag_of_words(sentiment_words, docs):
    """
    Generate a bag of words from documents for a certain sentiment

    Parameters
    ----------
    sentiment_words: Pandas Series
        Words that signify a certain sentiment
    docs : list of str
        List of documents used to generate bag of words

    Returns
    -------
    bag_of_words : 2-d Numpy Ndarray of int
        Bag of words sentiment for each document
        The first dimension is the document.
        The second dimension is the word.
    """
    
    # TODO: Implement
    vec = CountVectorizer(vocabulary=sentiment_words)
    vectors = vec.fit_transform(docs)
    words_list = vec.get_feature_names()
    bag_of_words = np.zeros([len(docs), len(words_list)])
    
    for i in range(len(docs)):
        bag_of_words[i] = vectors[i].toarray()[0]

    return bag_of_words.astype(int)


project_tests.test_get_bag_of_words(get_bag_of_words)

Using the `get_bag_of_words` function, we'll generate a bag of words for all the documents.

In [ ]:
sentiment_bow_ten_ks = {}

for ticker, ten_ks in ten_ks_by_ticker.items():
    lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
    
    sentiment_bow_ten_ks[ticker] = {
        sentiment: get_bag_of_words(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
        for sentiment in sentiments}


project_helper.print_ten_k_data([sentiment_bow_ten_ks[example_ticker]], sentiments)

### Jaccard Similarity
Using the bag of words, let's calculate the jaccard similarity on the bag of words and plot it over time. Implement `get_jaccard_similarity` to return the jaccard similarities between each tick in time. Since the input, `bag_of_words_matrix`, is a bag of words for each time period in order, you just need to compute the jaccard similarities for each neighboring bag of words. Make sure to turn the bag of words into a boolean array when calculating the jaccard similarity.

In [ ]:
from sklearn.metrics import jaccard_similarity_score


def get_jaccard_similarity(bag_of_words_matrix):
    """
    Get jaccard similarities for neighboring documents

    Parameters
    ----------
    bag_of_words : 2-d Numpy Ndarray of int
        Bag of words sentiment for each document
        The first dimension is the document.
        The second dimension is the word.

    Returns
    -------
    jaccard_similarities : list of float
        Jaccard similarities for neighboring documents
    """
    
    # TODO: Implement
    jaccard_similarities = []
    bag_of_words_matrix = np.array(bag_of_words_matrix, dtype=bool)
    
    for i in range(len(bag_of_words_matrix)-1):
            u = bag_of_words_matrix[i]
            v = bag_of_words_matrix[i+1]
            jaccard_similarities.append(jaccard_similarity_score(u,v))    
    
    return jaccard_similarities


project_tests.test_get_jaccard_similarity(get_jaccard_similarity)

Using the `get_jaccard_similarity` function, let's plot the similarities over time.

In [ ]:
# Get dates for the universe
file_dates = {
    ticker: [ten_k['file_date'] for ten_k in ten_ks]
    for ticker, ten_ks in ten_ks_by_ticker.items()}  

jaccard_similarities = {
    ticker: {
        sentiment_name: get_jaccard_similarity(sentiment_values)
        for sentiment_name, sentiment_values in ten_k_sentiments.items()}
    for ticker, ten_k_sentiments in sentiment_bow_ten_ks.items()}


project_helper.plot_similarities(
    [jaccard_similarities[example_ticker][sentiment] for sentiment in sentiments],
    file_dates[example_ticker][1:],
    'Jaccard Similarities for {} Sentiment'.format(example_ticker),
    sentiments)

### TFIDF
using the sentiment word lists, let's generate sentiment TFIDF from the 10-k documents. Implement `get_tfidf` to generate TFIDF from each document, using sentiment words as the terms. You can ignore words that are not in `sentiment_words`.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


def get_tfidf(sentiment_words, docs):
    """
    Generate TFIDF values from documents for a certain sentiment

    Parameters
    ----------
    sentiment_words: Pandas Series
        Words that signify a certain sentiment
    docs : list of str
        List of documents used to generate bag of words

    Returns
    -------
    tfidf : 2-d Numpy Ndarray of float
        TFIDF sentiment for each document
        The first dimension is the document.
        The second dimension is the word.
    """
    
    # TODO: Implement
    vec = TfidfVectorizer(vocabulary=sentiment_words)
    tfidf = vec.fit_transform(docs)
    
    return tfidf.toarray()


project_tests.test_get_tfidf(get_tfidf)

Using the `get_tfidf` function, let's generate the TFIDF values for all the documents.

In [ ]:
sentiment_tfidf_ten_ks = {}

for ticker, ten_ks in ten_ks_by_ticker.items():
    lemma_docs = [' '.join(ten_k['file_lemma']) for ten_k in ten_ks]
    
    sentiment_tfidf_ten_ks[ticker] = {
        sentiment: get_tfidf(sentiment_df[sentiment_df[sentiment]]['word'], lemma_docs)
        for sentiment in sentiments}

    
project_helper.print_ten_k_data([sentiment_tfidf_ten_ks[example_ticker]], sentiments)

### Cosine Similarity
Using the TFIDF values, we'll calculate the cosine similarity and plot it over time. Implement `get_cosine_similarity` to return the cosine similarities between each tick in time. Since the input, `tfidf_matrix`, is a TFIDF vector for each time period in order, you just need to computer the cosine similarities for each neighboring vector.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def get_cosine_similarity(tfidf_matrix):
    """
    Get cosine similarities for each neighboring TFIDF vector/document

    Parameters
    ----------
    tfidf : 2-d Numpy Ndarray of float
        TFIDF sentiment for each document
        The first dimension is the document.
        The second dimension is the word.

    Returns
    -------
    cosine_similarities : list of float
        Cosine similarities for neighboring documents
    """
    
    # TODO: Implement
    cosine_similarities = []    
    
    for i in range(len(tfidf_matrix)-1):
        cosine_similarities.append(cosine_similarity(tfidf_matrix[i].reshape(1, -1),tfidf_matrix[i+1].reshape(1, -1))[0,0])
    
    return cosine_similarities


project_tests.test_get_cosine_similarity(get_cosine_similarity)

Let's plot the cosine similarities over time.

In [ ]:
cosine_similarities = {
    ticker: {
        sentiment_name: get_cosine_similarity(sentiment_values)
        for sentiment_name, sentiment_values in ten_k_sentiments.items()}
    for ticker, ten_k_sentiments in sentiment_tfidf_ten_ks.items()}


project_helper.plot_similarities(
    [cosine_similarities[example_ticker][sentiment] for sentiment in sentiments],
    file_dates[example_ticker][1:],
    'Cosine Similarities for {} Sentiment'.format(example_ticker),
    sentiments)

## Evaluate Alpha Factors
Just like we did in project 4, let's evaluate the alpha factors. For this section, we'll just be looking at the cosine similarities, but it can be applied to the jaccard similarities as well.
### Price Data
Let's get yearly pricing to run the factor against, since 10-Ks are produced annually.

In [ ]:
pricing = pd.read_csv('../../data/project_5_yr/yr-quotemedia.csv', parse_dates=['date'])
pricing = pricing.pivot(index='date', columns='ticker', values='adj_close')

pricing

### Dict to DataFrame
The alphalens library uses dataframes, so we we'll need to turn our dictionary into a dataframe. 

In [ ]:
cosine_similarities_df_dict = {'date': [], 'ticker': [], 'sentiment': [], 'value': []}


for ticker, ten_k_sentiments in cosine_similarities.items():
    for sentiment_name, sentiment_values in ten_k_sentiments.items():
        for sentiment_values, sentiment_value in enumerate(sentiment_values):
            cosine_similarities_df_dict['ticker'].append(ticker)
            cosine_similarities_df_dict['sentiment'].append(sentiment_name)
            cosine_similarities_df_dict['value'].append(sentiment_value)
            cosine_similarities_df_dict['date'].append(file_dates[ticker][1:][sentiment_values])

cosine_similarities_df = pd.DataFrame(cosine_similarities_df_dict)
cosine_similarities_df['date'] = pd.DatetimeIndex(cosine_similarities_df['date']).year
cosine_similarities_df['date'] = pd.to_datetime(cosine_similarities_df['date'], format='%Y')


cosine_similarities_df.head()

### Alphalens Format
In order to use a lot of the alphalens functions, we need to aligned the indices and convert the time to unix timestamp. In this next cell, we'll do just that.

In [ ]:
import alphalens as al


factor_data = {}
skipped_sentiments = []

for sentiment in sentiments:
    cs_df = cosine_similarities_df[(cosine_similarities_df['sentiment'] == sentiment)]
    cs_df = cs_df.pivot(index='date', columns='ticker', values='value')
    
    try:
        data = al.utils.get_clean_factor_and_forward_returns(cs_df.stack(), pricing.loc[cs_df.index], quantiles=5, bins=None, periods=[1])
        factor_data[sentiment] = data
    except:
        skipped_sentiments.append(sentiment)

if skipped_sentiments:
    print('\nSkipped the following sentiments:\n{}'.format('\n'.join(skipped_sentiments)))
factor_data[sentiments[0]].head()

### Alphalens Format with Unix Time
Alphalen's `factor_rank_autocorrelation` and `mean_return_by_quantile` functions require unix timestamps to work, so we'll also create factor dataframes with unix time.

In [ ]:
unixt_factor_data = {
    factor: data.set_index(pd.MultiIndex.from_tuples(
        [(x.timestamp(), y) for x, y in data.index.values],
        names=['date', 'asset']))
    for factor, data in factor_data.items()}

### Factor Returns
Let's view the factor returns over time. We should be seeing it generally move up and to the right.

In [ ]:
ls_factor_returns = pd.DataFrame()

for factor_name, data in factor_data.items():
    ls_factor_returns[factor_name] = al.performance.factor_returns(data).iloc[:, 0]

(1 + ls_factor_returns).cumprod().plot()

### Basis Points Per Day per Quantile
It is not enough to look just at the factor weighted return. A good alpha is also monotonic in quantiles. Let's looks the basis points for the factor returns.

In [ ]:
qr_factor_returns = pd.DataFrame()

for factor_name, data in unixt_factor_data.items():
    qr_factor_returns[factor_name] = al.performance.mean_return_by_quantile(data)[0].iloc[:, 0]

(10000*qr_factor_returns).plot.bar(
    subplots=True,
    sharey=True,
    layout=(5,3),
    figsize=(14, 14),
    legend=False)

### Turnover Analysis
Without doing a full and formal backtest, we can analyze how stable the alphas are over time. Stability in this sense means that from period to period, the alpha ranks do not change much. Since trading is costly, we always prefer, all other things being equal, that the ranks do not change significantly per period. We can measure this with the **Factor Rank Autocorrelation (FRA)**.

In [ ]:
ls_FRA = pd.DataFrame()

for factor, data in unixt_factor_data.items():
    ls_FRA[factor] = al.performance.factor_rank_autocorrelation(data)

ls_FRA.plot(title="Factor Rank Autocorrelation")

### Sharpe Ratio of the Alphas
The last analysis we'll do on the factors will be sharpe ratio. Let's see what the sharpe ratio for the factors are. Generally, a Sharpe Ratio of near 1.0 or higher is an acceptable single alpha for this universe.

In [ ]:
daily_annualization_factor = np.sqrt(252)

(daily_annualization_factor * ls_factor_returns.mean() / ls_factor_returns.std()).round(2)

That's it! You've successfully done sentiment analysis on 10-ks!
## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.